In [128]:
import re
from fuzzywuzzy import fuzz

dictlist = {}
indexlist = {}
entrylen = {}
indexhead = {}

biof = open('/Users/gcrane/Desktop/OldMacintoshHD/sgml/texts/Smith/bio-pn2.xml')
inenty = 0
curentry = ''
for l in biof:
    m = re.search('type="entry" id="([^"]+)',l)
    if(m):
        curentry = m[1]
        l = re.sub("([a-zA-Z])'([a-z])",'\g<1>\g<2>',l)

        workl = re.sub('</head>.+','',l)
        workl = re.sub('\n','',l)
        indexhead[curentry] = re.sub('<[^<]+>','',workl)

    m = re.search('michael',curentry)
    if(not m):
        inentry = 0
    else:
        inentry = 1
    if(inentry):
        l = re.sub('\&eacute;','é',l)
        l = re.sub('\&ouml;','ö',l)
        l = re.sub('\&uuml;','ü',l)
        l = re.sub('<[^>]+>','',l)
        l = re.sub('[,;:!"\.\(\)]',' ',l)
        l = re.sub("'s\\b",'',l)
        l = re.sub("([a-zA-Z])'([a-z])",'\g<1>\g<2>',l)
        if(curentry in entrylen):
            entrylen[curentry] = entrylen[curentry] + len(l)
        else:
            entrylen[curentry] = len(l)
            
        words = l.split()
        #print(curentry,l,end='')
        if(not curentry in dictlist):
            dictlist[curentry] = {}
        for i in range(0,len(words)):
            #if(len(words[i]) < 3):
            #    continue
            if(words[i][0].isupper()):
                workword = words[i].lower()
                workword = re.sub('constantinus','constantine',workword)
               # print(words[i])
                if(workword in dictlist[curentry]):
                    dictlist[curentry][workword] = dictlist[curentry][workword] + 1
                else:
                    dictlist[curentry][workword] =  1                    
                #if(i < len(words)-1 and words[i+1][0].isupper()):
                #   print(words[i],words[i+1])
        

biof.close()

for foo in sorted(entrylen,key=entrylen.get,reverse=True):
    continue
    print(foo,len(dictlist[foo]))
    #for boo in dictlist[foo]:
    #    print(boo,dictlist[foo][boo])
        

f = open('gibbon-bury-newyork1906-index-ner.xml')

indexentrylen = {}
inentry = 0
for l in f:
    l = re.sub('\n+$','',l)
    m = re.search('type="entry" xml:id="([^"]+)"',l)
    if(m):
        indexentry = m[1]
    if(re.search('<head>',l)):
        if(re.search('<head><pers.+Michael\\b',l)):
            inentry = 1
            indexhead[indexentry] = re.sub('<[^>]+>','',l)
            #print(indexentry,indexhead[indexentry])

        else:
            inentry = 0
            
    if(not inentry):
        continue
        
    l,entrylen = re.subn('<ref[^>]+>[^<]+','',l)
    l = re.sub('<[^>]+>','',l)
    
    if(entrylen):
        if( indexentry in indexentrylen):
            indexentrylen[indexentry] = indexentrylen[indexentry] + entrylen
        else:
            indexentrylen[indexentry] = entrylen
    l = re.sub('\&eacute;','é',l)
    l = re.sub('\&ouml;','ö',l)
    l = re.sub('\&uuml;','ü',l)
    l = re.sub('<[^>]+>','',l)
    l = re.sub('[,;:!"\.\(\)]',' ',l)
    l = re.sub("'s\\b",'',l)
    l = re.sub("([a-zA-Z])'([a-z])",'\g<1>\g<2>',l)

    words = l.split()
        #print(curentry,l,end='')
    if(not indexentry in indexlist):
        indexlist[indexentry] = {}
    for i in range(0,len(words)):
        #if(len(words[i]) < 3):
       #     continue
        if(words[i][0].isupper()):
            workword = words[i].lower()
               # print(words[i])
            if(workword in indexlist[indexentry]):
                indexlist[indexentry][workword] = indexlist[indexentry][workword] + 1
            else:
                indexlist[indexentry][workword] =  1                    
                #if(i < len(words)-1 and words[i+1][0].isupper()):
                #   print(words[i],words[i+1])
   
    #print(curentry,l,end='')
f.close()

def sharednames(l1,l2):
    hits = 0
    hitlist = []
    tries = 0
    for foo in l1:
        if(tries > 10):
            continue
        for boo in l2:
            #print('comping',foo,boo)
            if(foo == boo):
                hits = hits + 1
                hitlist.append(foo)
        tries = tries + 1
    return(hitlist)

def bestfit(curindex):
    bestsofar = 0
    print('curindex',curindex,indexlist[curindex])
    for foo in dictlist:
        sharedl = sharednames(dictlist[foo],indexlist[curindex])
        t1 = indexhead[curindex].lower()
        t1 = re.sub('\.','',t1)
        if( re.search('\s+or\s+',t1)):
            t1,t1b = t1.split(' or ')
        else:
            t1b = ''
        t2 = re.sub('-',' ',foo)
        t2 = re.sub('constantinus','constantine',t2)
        t2 = re.sub('[\-0-9]+bio[\-0-9]*','',t2)
        if( re.search(' ',t1) and re.search(t1 + ' ',t2)):
            print('gotit',t1,t2,foo)
            return
        if( t1b and  re.search(' ',t1b) and re.search(t1b + ' ',t2)):
            print('gotit',t1b,t2,foo)
            return
        curfuzz = fuzz.ratio(t1,t2)
        if(t1b and fuzz.ratio(t1b,t2) > curfuzz):
            curfuzz = fuzz.ratio(t1b,t2)
        #print(sharedl,indexhead[curindex],foo,fuzz.ratio(t1,t2),'\n')
        #if(fuzz.ratio(t1,t2)> bestsofar):
        #    bestsofar = fuzz.ratio(t1,t2)
        if(len(sharedl)> bestsofar):
            bestsofar = len(sharedl)
            besthit = foo
            bestshared = sharedl
    if(bestsofar < 2):
        print('no good match')
    else:
        print('best',bestsofar,bestshared,besthit,indexhead[curindex])
    print('smith',besthit,dictlist[besthit])
    #print('bury',curindex,indexhead[curindex],indexlist[curindex])

#bestfit('ent1791')
#for foo in sorted(indexentrylen,key=indexentrylen.get,reverse=True):
for foo in indexentrylen:  
    print('\n\n',foo,indexentrylen[foo],indexhead[foo])
    rval = bestfit(foo)




 ent1503 3 Michael Cerularius
curindex ent1503 {'michael': 1, 'cerularius': 1, 'patriarch': 1, 'constantinople': 1, 'pope': 1}
best 3 ['michael', 'cerularius', 'constantinople'] michael-bio-7 Michael Cerularius
smith michael-bio-7 {'michael': 2, 'cerularius': 4, 'constantinople': 2, 'latin': 3, 'leo': 3, 'ix': 2, 'cardinals': 1, 'humbert': 2, 'frederic': 1, 'peter': 1, 'amalfi': 1, 'pope': 1, 'stratioticus': 1, 'isaac': 3, 'comnenus': 1, 'i': 2, 'works': 1, 'decisio': 1, 'synodica': 1, 'nuptüs': 1, 'septimo': 1, 'gradu': 1, 'editions': 6, 'greek': 2, 'leunclavius': 2, 'jus': 2, 'graeco-roman': 2, 'de': 6, 'matrimonio': 1, 'epistolae': 1, 'ii': 1, 'petrum': 1, 'antiochenum': 1, 'cotelerius': 2, 'eccles': 2, 'graec': 2, 'monument': 1, 'sacerdotis': 1, 'uxore': 1, 'adulterio': 1, 'patres': 1, 'apotl': 1, 'edictum': 1, 'synodale': 1, 'latinos': 1, 'pittacia': 1, 'excommunicatione': 1, 'latinis': 1, 'legatis': 1, 'legatos': 1, 'junü': 1, 'graece': 2, 'latine': 2, 'allatius': 1, 'libr': 1,

In [135]:
#sharednames(dictlist['constantinus-bio-2'],indexlist['ent1804'])

ambnames = {}
for foo in dictlist:
    print(foo,indexhead[foo])
    words = indexhead[foo].split()
    for word in words:
        if(word in ambnames):
            ambnames[word].append(foo)
        else:
            ambnames[word] = [foo]

for foo in sorted(ambnames):
    print(foo,ambnames[foo])
    

anchialus-michael-bio-1 Anchialus, Michael
attaliata-michael-bio-1 ATTALI'ATA* 
ducas-michael-bio-1 Ducas, Michael
glycas-michael-bio-1 Glycas, Michael
michael-i-rhangabe-bio-1 Michael I. or Michael Rhangabe
michael-ii-balbus-bio-1 Michael Ii. or Michael Balbus
michael-iii-bio-1 Michael Iii.
michael-iv-paphlago-bio-1 Michael Iv. or Michael Paphlago
michael-v-calapiiates-bio-1 Michael V. or Michael Calapiiates
michael-vi-stratioticus-bio-1 Michael Vi. or Michael Stratioticus
michael-vii-ducas-parapinaces-bio-1 Michael Vii. or Michael Ducas Parapinaces
michael-viii-palaeologus-bio-1 Michael Viii. or Michael Palaeologus
michael-ix-palaeologus-bio-1 Michael Ix. or Michael Palaeologus
michael-bio-1 MICHAEL
michael-bio-2 MICHAEL
michael-bio-3 MICHAEL
michael-bio-4 MICHAEL
michael-bio-5 MICHAEL
michael-bio-6 MICHAEL
michael-bio-7 MICHAEL
michael-bio-8 MICHAEL
michael-bio-9 MICHAEL
michael-bio-10 MICHAEL
michael-bio-11 MICHAEL
michael-bio-12 MICHAEL
michael-bio-13 MICHAEL
michael-bio-14 MICHAE

In [115]:
indexlist['ent1804']

{'constantine': 1, 'britain': 1, 'vienna': 1, 'arles': 1}